# HMS-Commander
Author: William (Bill) Katzenmeyer, P.E., C.F.M. (C.H. Fenstermaker and Associates, LLC)

Source: https://github.com/billk-FM/HEC-Commander-Tools

In [ ]:
# User Defined Inputs

# Provide the HMS Project Directory and Project Name
hms_project_directory = r"C:\Your_HMS_Project_Directory"
hms_basin_file = "Your_HMS_Basin.basin"

# Define Calibration Runs CSV Filename (place in HMS Project Directory)
user_calibration_runs_csv_filename = "Example_Test.csv"

hms_run_names = [
    "Your_Run_Name_1",
]

# If multiple run names are provided, each will generate a full set of user-defined runs

#Set a DSS output file suffix to differentiate run sets
hms_dss_suffix = "_Batch_Name"  # define your suffix here
print("DSS Suffix: " + hms_dss_suffix)

# Override Baseflow:None to Baseflow: Recession
hms_recession_baseflow = True

In [ ]:
# ---- Additional Settings ----

# Override Baseflow None to Baseflow Recession
Baseflow_Method_Set_to_Recession = "Yes"

# Define the path to the HEC-HMS executable
hms_executable_path = r"C:\Program Files\HEC\HEC-HMS\4.9"
print("HEC-HMS Executable Path: " + hms_executable_path)

# Define paths for HMScompute.py and HMScompute.bat which are used to run HMS through Jython
hms_compute_py_path = r"C:\jython2.7.3\HMScompute.py"
hms_compute_bat_path = r"C:\jython2.7.3\HMScompute.bat"

''' If jython heap size is too small, the HMS will freeze at 100% CPU and never finish'''
jython_initial_heap_size = "256m"       #initial heap size for java virtual machine
print("Jython Initial Heap Size: " + jython_initial_heap_size)
jython_maximum_heap_size = "4096m"      #maximum heap size for java virtual machine
print("Jython Maximum Heap Size: " + jython_maximum_heap_size)

# Set Canopy Method to Simple Zero to avoid warnings (Deprecated, not needed after HMS 4.9)
Canopy_Method_Set_To_Simple_Zero = "No"

In [ ]:
# Install/Import Required Python Packages
import sys
import subprocess

# List of packages you want to ensure are installed
packages = ["os", "shutil", "pandas", "geopandas", "subprocess", "re", "csv", "itertools"]

# Logic to Install Packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    print("Installed " + package + " successfully")

for package in packages:
    try:
        # If the import succeeds, the package is installed, so we can move on
        __import__(package)
    except ImportError:
        # If the import fails, the package is not installed and we try to install it
        install(package)

# Import All Required Libraries for Script
import shutil
import pandas as pd
import os
import time
import itertools
import re
import csv
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# Check for Jython 2.7.3 at C:\jython2.7.3

jython_path = r"C:\jython2.7.3"
jython_jar_path = os.path.join(jython_path, "jython.jar")

def jython_exists(path):
    exists = os.path.exists(path)
    return exists

def print_installation_instructions():
    print("Jython is not found at the provided path.")
    print("Please install the necessary software:")
    print("1. Java: https://www.java.com/en/download/")
    print("2. Java SE Development Kit (check for the latest version): https://www.oracle.com/java/technologies/javase-jdk-downloads.html")
    print("3. Jython: https://www.jython.org/download.html")

if jython_exists(jython_jar_path):
    print(f"Jython Exists at {jython_jar_path}")
else:
    print_installation_instructions()

In [ ]:
# Build HMScompute.bat and HMScompute.py for Jython 2.7.3


# Define the path and content for HMScompute.py
hms_compute_py_content = '''from hms.model import Project
from hms import Hms

import glob
import sys

hmspth = sys.argv[1]
runName = sys.argv[2]
# print 'running' + str(hmspth)

myProject = Project.open(hmspth)
myProject.computeRun(runName)
myProject.close()

Hms.shutdownEngine()
'''

# Define the path and content for HMScompute.bat
hms_compute_bat_content = f'''set "HMS={hms_executable_path}"
set "PATH=%HMS%\\bin\\gdal;%PATH%"
set "GDAL_DRIVER_PATH=%HMS%\\bin\\gdal\\gdalplugins"
set "GDAL_DATA=%HMS%\\bin\\gdal\\gdal-data"
set "PROJ_LIB=%HMS%\\bin\\gdal\\projlib"

set "CLASSPATH=%HMS%\\hms.jar;%HMS%\\lib\\*"
C:\\jython2.7.3\\bin\\jython.exe -J-Xms{jython_initial_heap_size} -J-Xmx{jython_maximum_heap_size} -Djava.library.path="%HMS%\\bin;%HMS%\\bin\\gdal" {hms_compute_py_path} %1 %2
'''

# Write the content for HMScompute.py
with open(hms_compute_py_path, 'w') as py_file:
    py_file.write(hms_compute_py_content)
print(f"Wrote content to {hms_compute_py_path}")


# Write the content for HMScompute.bat
with open(hms_compute_bat_path, 'w') as bat_file:
    bat_file.write(hms_compute_bat_content)
print(f"Wrote content to {hms_compute_bat_path}")

In [ ]:
#Logic to determine HMS Project Name

# Get HMS Project Name from hms_project_directory
def get_hms_project_name(hms_project_directory):
    """
    Given an HMS project directory, return the HMS project name.
    The project name is determined based on the .hms file in the directory.
    If more than one .hms file exists or none exist, an appropriate error message is returned.

    Args:
    - hms_project_directory (str): Path to the HMS project directory

    Returns:
    - str: HMS project name or error message
    """

    # List all files in the directory
    all_files = os.listdir(hms_project_directory)

    # Filter for files with .hms extension
    hms_files = [f for f in all_files if f.endswith('.hms')]

    # Check if there's exactly one .hms file
    if len(hms_files) == 1:
        # Extract the project name by stripping the .hms extension
        return hms_files[0].replace('.hms', '')
    elif len(hms_files) > 1:
        return "Error: More than one .hms file exists in the directory!"
    else:
        return "Error: No .hms file found in the directory!"

# Print Project Name or Error
hms_project_name = get_hms_project_name(hms_project_directory)
print("HMS Project Name: ", hms_project_name)

In [ ]:
# Calculate and Print necessary paths and file names

# Calculated Paths and File Names (.run, .basin, backup paths)
user_calibration_runs_csv_fullpath = os.path.join(hms_project_directory, user_calibration_runs_csv_filename)
hms_project_run_file = f"{hms_project_name}.run"
hms_project_run_file_path = os.path.join(hms_project_directory, hms_project_run_file)
hms_grid_file = f"{hms_project_name}.grid"
hms_basin_file_path = os.path.join(hms_project_directory, hms_basin_file)
hms_project_run_file_backup_path = hms_project_run_file_path + ".bak"
hms_basin_file_backup_path = hms_basin_file_path + ".bak"

hms_grid_file_path = os.path.join(hms_project_directory, hms_grid_file)
hms_grid_file_backup_path = hms_grid_file_path + ".bak"

# Print statements
print("HMS User Calibration Runs CSV: ", user_calibration_runs_csv_fullpath)
print("HMS Project Run File:", hms_project_run_file)
print("HMS Project Run File Path:", hms_project_run_file_path)
print("HMS Basin File:", hms_basin_file)
print("HMS Basin File Path:", hms_basin_file_path)
print("HMS Project Run File Backup Path:", hms_project_run_file_backup_path)
print("HMS Basin File Backup Path:", hms_basin_file_backup_path)

In [ ]:
# Create Lock File and Logic for Working File Backup and Restore
# Define the path for the lock file based on one of the .bak file paths
lock_file_directory = os.path.dirname(hms_project_run_file_backup_path)
lock_file_path = os.path.join(lock_file_directory, "HMSCommander.lock")

# Step 1: Check if HMSCommander.lock exists
if os.path.exists(lock_file_path):
    print("Lock file exists. Restoring files from .bak backup.")

    # Step 2: Restore files from backup
    for backup, original in [(hms_project_run_file_backup_path, hms_project_run_file_path),
                             (hms_basin_file_backup_path, hms_basin_file_path),
                             (hms_grid_file_backup_path, hms_grid_file_path)]:
        try:
            if os.path.exists(backup):
                shutil.copy(backup, original)
                # Delete the .bak file
                os.remove(backup)
                print(f"Restored and deleted file {backup}")
            else:
                print(f"Backup file {backup} does not exist. Skipping restore.")
        except Exception as e:
            print(f"Error restoring file from {backup} to {original}: {e}")

    # Step 3: Delete the lock file
    os.remove(lock_file_path)
else:
    print("Lock file does not exist. Proceeding to backup files.")

# Step 4: Backup files
for original, backup in [(hms_project_run_file_path, hms_project_run_file_backup_path),
                         (hms_basin_file_path, hms_basin_file_backup_path),
                         (hms_grid_file_path, hms_grid_file_backup_path)]:
    try:
        if os.path.exists(original):
            shutil.copy(original, backup)
            print(f"Backed up file from {original} to {backup}")
        else:
            print(f"Original file {original} does not exist. Skipping backup.")
    except Exception as e:
        print(f"Error backing up file from {original} to {backup}: {e}")

# Step 5: Create a new HMSCommander.lock file
with open(lock_file_path, 'w') as f:
    f.write("Lock file for HMS Commander operations.")

print("Lock file created.")

In [ ]:
# Read run parameters from CSV file
user_calibration_df = pd.read_csv(user_calibration_runs_csv_fullpath, dtype={'user_run_number_from_csv': int})

def load_user_calibration_csv_data(file_path):
    user_calibration_df = pd.read_csv(file_path, dtype={'user_run_number_from_csv': int})  # Directly reading csv using pandas to return DataFrame
    return user_calibration_df # Show DataFrame

load_user_calibration_csv_data(user_calibration_runs_csv_fullpath)

In [ ]:
# Define all necessary functions

# Reads the content of the HMS basin file
def read_basin_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content


#Updates the grid file with the new DSS file path based on the impervious area scale.
def update_impervious_grid_definitions(grid_file_path, grid_def, dss_file_path, impervious_area_scale):
    
    with open(grid_file_path, 'r') as file:
        lines = file.readlines()

    grid_def_found = False
    for i, line in enumerate(lines):
        if f"Grid: {grid_def}" in line:
            grid_def_found = True
            print("Grid Definition Found!")
        if grid_def_found and "DSS File Name:" in line:
            old_dss_file_name = line.split(": ")[1].strip()
            new_dss_file_name = "data\\" + dss_file_path.split("\\")[-1]
            lines[i] = line.replace(old_dss_file_name, new_dss_file_name)
            print("DSS FileName Replaced!")
            break

    with open(grid_file_path, 'w') as file:
        file.writelines(lines)


# Optional: pauses the script execution until the user provides input.
def pause_script():
    
    user_input = input("Press Enter to continue or 'q' to quit: ")
    if user_input.lower() == 'q':
        exit()


# Optional: print the first 60 lines of a file
def print_first_60_lines(file_path):
    """Prints the first 60 lines of a specified file."""
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if i < 60:
                print(line.strip())
            else:
                break


# Loads the user calibration data from the specified file path into a pandas dataframe.
def load_subbasin_tc_data_to_pandas_dataframe(filepath):
    """Loads subbasin time of concentration and storage coefficient data into a pandas dataframe."""
    with open(filepath, 'r') as file:
        lines = file.readlines()

    subbasin_data = []
    current_subbasin = None
    toc = None
    sc = None

    for line in lines:
        if 'Subbasin:' in line:
            current_subbasin = line.split(': ')[1].strip()
        elif 'Time of Concentration:' in line:
            toc = float(line.split(': ')[1].strip())
        elif 'Storage Coefficient:' in line:
            sc = float(line.split(': ')[1].strip())
        elif 'End:' in line:
            if current_subbasin and toc and sc:
                subbasin_data.append([current_subbasin, toc, sc])
                current_subbasin, toc, sc = None, None, None

    subbasin_tc_r = pd.DataFrame(subbasin_data, columns=['subbasin_name', 'time_of_concentration', 'storage_coefficient'])
    return subbasin_tc_r

# Loads the user calibration data for the current run number into a pandas dataframe.
def fetch_calibration_parameters_for_current_run(user_calibration_df, current_run_number):
    """Fetches the calibration parameters for a specified run number."""
    current_calibration = user_calibration_df[user_calibration_df['user_run_number_from_csv'] == current_run_number]
    if not current_calibration.empty:
        return current_calibration.iloc[0]  
    else:
        print("No match found for current run number in user calibration data.")
        return None  # Return None if no matching calibration is found


# Scales the time of concentration and storage coefficient data using the provided calibration parameters.
def scale_tcr_by_run_scale_factor(subbasin_tc_df, calibration_parameters):
    """Scales the time of concentration and storage coefficient data using the provided calibration parameters."""
    if calibration_parameters is not None:
        subbasin_tc_df['time_of_concentration_scaled'] = (subbasin_tc_df['time_of_concentration'] * calibration_parameters['time_of_concentration_scale']).round(2)
        subbasin_tc_df['storage_coefficient_scaled'] = (subbasin_tc_df['storage_coefficient'] * calibration_parameters['storage_coefficient_scale']).round(2)
        return subbasin_tc_df
    else:
        print("No calibration parameters provided. Unable to scale data.")
        return pd.DataFrame()


# Writes the scaled time of concentration and storage coefficient data back to the basin file.
def write_subbasin_tc_data_to_basin_file(filepath, subbasin_tc_r):
    """Writes the scaled time of concentration and storage coefficient data back to the basin file."""
    with open(filepath, 'r') as file:
        lines = file.readlines()

    new_lines = []
    subbasin_counter = 0
    for line in lines:
        line = line.strip()
        if line.startswith('Subbasin: '):
            current_subbasin = line.split(': ')[1]
            
            # Increase the subbasin counter
            subbasin_counter += 1
            
            # Print output only for the first 2 subbasins
            if subbasin_counter <= 2:
                print(f'Processing subbasin: {current_subbasin}')
        elif line.startswith('Time of Concentration: '):
            old_toc = line.split(': ')[1]
            toc_scale = subbasin_tc_r.loc[subbasin_tc_r['subbasin_name'] == current_subbasin, 'time_of_concentration_scaled'].item()
            line = f"Time of Concentration: {toc_scale}"
            
            # Print output only for the first 5 subbasins
            if subbasin_counter <= 5:
                print(f'For subbasin {current_subbasin}, Time of Concentration was changed from {old_toc} to {toc_scale}')
        elif line.startswith('Storage Coefficient: '):
            old_sc = line.split(': ')[1]
            sc_scale = subbasin_tc_r.loc[subbasin_tc_r['subbasin_name'] == current_subbasin, 'storage_coefficient_scaled'].item()
            line = f"Storage Coefficient: {sc_scale}"
            
            # Print output only for the first 5 subbasins
            if subbasin_counter <= 5:
                print(f'For subbasin {current_subbasin}, Storage Coefficient was changed from {old_sc} to {sc_scale}')
        new_lines.append(line + '\n')
    
    with open(filepath, 'w') as file:
        print("Writing TC and R to .basin file")
        file.writelines(new_lines)


#Finds the path of the Impervious DSS file based on the impervious area scale.
def find_impervious_dss_grids(directory, impervious_area_scale):
    
    scale_factor_to_filename = {
        1.2: "Impervious_1.2_SF.dss",
        1.4: "Impervious_1.4_SF.dss",
        1.6: "Impervious_1.6_SF.dss",
        1.8: "Impervious_1.8_SF.dss",
        2.0: "Impervious_2.0_SF.dss",
    }
    filename = scale_factor_to_filename.get(impervious_area_scale)
    if filename:
        full_path = os.path.join(directory, "data", filename)
        if os.path.isfile(full_path):
            return full_path
        else:
            print(f"Error: File '{filename}' cannot be found in the directory '{directory}/data'.")
            return None
    else:
        print(f"Error: No corresponding dss file found for the impervious area scale '{impervious_area_scale}'.")
        return None

# Updates the basin file with new soil and baseflow parameters
def process_soil_and_baseflow_to_basin_file(file_path, values):
    # Your original search_replace dictionary
    search_replace = {
        r"Initial Deficit Scale:\s*([\d.]+)": f"Initial Deficit Scale: {values['initial_deficit_scale']}",
        r"Maximum Deficit Scale:\s*([\d.]+)": f"Maximum Deficit Scale: {values['maximum_deficit_scale']}",
        r"Percolation Rate Scale:\s*([\d.]+)": f"Percolation Rate Scale: {values['percolation_rate_scale']}",
        r"Impervious Area Scale:\s*([\d.]+)": f"Impervious Area Scale: {values['impervious_area_scale']}" if values['impervious_area_scale'] <= 1.0 else f"Impervious Area Scale: 1.0",
        r"Recession Factor:\s*([\d.]+)": f"Recession Factor: {values['recession_factor']}",
        r"Initial Flow/Area Ratio:\s*([\d.]+)": f"Initial Flow/Area Ratio: {values['initial_flow_area_ratio']}",
        r"Threshold Flow to Peak Ratio:\s*([\d.]+)": f"Threshold Flow to Peak Ratio: {values['threshold_flow_to_peak_ratio']}"
    }

    # Read and modify the file line-by-line
    with open(file_path, "r") as file:
        lines = file.readlines()

    modified_lines = []
    for line in lines:
        modified_line = line
        for pattern, replacement in search_replace.items():
            match = re.search(pattern, line, flags=re.IGNORECASE)
            if match:
                old_value = match.group(1)
                modified_line = re.sub(pattern, replacement, line, flags=re.IGNORECASE)
                print(f'Replacing {pattern} in {file_path} from {old_value} to {replacement.split(": ")[1]}')
                break
        modified_lines.append(modified_line)

    # Write the modified content back to the file
    with open(file_path, "w") as file:
        file.writelines(modified_lines)

def update_baseflow_recession():
    """
    Updates the baseflow in hms_basin_file_data to Recession  Baseflow Method if set to "None".

    Parameters:
    - hms_basin_file_data: The initial basin file data
    - run: Dictionary containing the parameters 'recession_factor', 'initial_flow_area_ratio', and 'threshold_flow_to_peak_ratio'
    - hms_basin_file_path: Path to the basin file
    
    Returns:
    - Modified hms_basin_file_data
    """
    with open(hms_basin_file_path, 'r') as file:
        hms_basin_file_data = file.read()

    # Fill in Recession Baseflow if set as "None" and add required variables
    baseflow_none_text = "Baseflow: None"
    if baseflow_none_text in hms_basin_file_data:
        baseflow_text = "Baseflow: Recession\n Recession Factor: {}\n Initial Flow/Area Ratio: {}\n Threshold Flow to Peak Ratio: {}"
        baseflow_text = baseflow_text.format(run['recession_factor'], run['initial_flow_area_ratio'], run['threshold_flow_to_peak_ratio'])
        hms_basin_file_data = hms_basin_file_data.replace(baseflow_none_text, baseflow_text)
        print("Baseflow Set to None.  Inserting Recession Baseflow Parameters")  # Indicate that Baseflow data was filled

    with open(hms_basin_file_path, 'w') as file:
        file.write(hms_basin_file_data)
    return hms_basin_file_data



def modify_canopy_method(file_path):
    # Initialize an empty list to hold the lines of the file
    modified_lines = []

    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()


    # If the flag is True, perform the replacement

    lines_to_replace = '''\
    Canopy: None
    Allow Simultaneous Precip Et: No
    Plant Uptake Method: None
'''
    lines_to_insert = '''\
    Canopy: Simple
    Allow Simultaneous Precip Et: No
    Plant Uptake Method: None
    Initial Canopy Storage Percent: 1
    Canopy Storage Capacity: 0
    Crop Coefficient: 1.0
    End Canopy:
'''

    # Replace occurrences
    joined_lines = ''.join(lines)
    modified_content = joined_lines.replace(lines_to_replace, lines_to_insert)

    # Write back the modified content to the file
    with open(file_path, 'w') as file:
        file.write(modified_content)

In [ ]:
# Main Logic of Script to Run HMS for each Event and Calibration Run


# For each run_name in hms_run_names:
for run_name in hms_run_names:
    hms_run_name = run_name  # Set the current run name
    user_calibration_df['user_run_number_from_csv'] = user_calibration_df['user_run_number_from_csv'].astype(int)

    # For each calibration run, the script will:
    for _, run in user_calibration_df.iterrows():
        
        run = run.rename({k: k.lower() for k in run.index})
        current_run_number = int(run['user_run_number_from_csv'])

        print(f"Processing run number: {current_run_number}")  # Indicate which run is being processed

        hms_run_output_dss = f"{hms_run_name}_HMS_Run_{current_run_number}{hms_dss_suffix}.dss"
        hms_run_output_dss_path = os.path.join(hms_project_directory, f"{hms_run_output_dss}")

        # Backup the .grid file
        shutil.copy(hms_grid_file_path, hms_grid_file_backup_path)
        print("Backup of .grid file created.")  # Indicate that backup of .grid file has been made

        if not os.path.isfile(hms_run_output_dss_path):
            print("Output DSS file does not exist. Preparing HMS Run.")  

            # Update the .run file with the new DSS file name
            with open(hms_project_run_file_path, 'r') as file:
                run_data = file.readlines()

            run_found = False
            for i, line in enumerate(run_data):
                if f"Run: {hms_run_name}" in line:
                    run_found = True
                if "End:" in line:
                    run_found = False
                if run_found and line.strip().startswith("DSS File:"):
                    old_dss_file_name = line.split(":")[1].strip()
                    run_data[i] = line.replace(old_dss_file_name, hms_run_output_dss)
            print(f"Output DSS file name changed from {old_dss_file_name} to {hms_run_output_dss} in .run file.")

            # Write the updated .run data
            with open(hms_project_run_file_path, 'w') as file:
                file.writelines(run_data)

            # Load subbasin TC data and scale it
            subbasin_tc_r = load_subbasin_tc_data_to_pandas_dataframe(hms_basin_file_path)
            calibration_parameters = fetch_calibration_parameters_for_current_run(user_calibration_df, current_run_number)
            scaled_subbasin_tc_data = scale_tcr_by_run_scale_factor(subbasin_tc_r, calibration_parameters)

            if not scaled_subbasin_tc_data.empty:
                # Write back scaled 'time_of_concentration' and 'storage_coefficient' to the .basin file
                write_subbasin_tc_data_to_basin_file(hms_basin_file_path, scaled_subbasin_tc_data)

            # The main logic to check the condition before updating baseflow recession parameters
            if hms_recession_baseflow and Baseflow_Method_Set_to_Recession == "Yes":
                # Assuming you have already defined and set the variables hms_basin_file_data, run, and hms_basin_file_path
                update_baseflow_recession()
            else:
                print("Baseflow already set, or Baseflow_Method_Set_to_Recession is not set to 'Yes'. No changes made.")

            # Update the .basin file with the new soil and recession baseflow parameters 
            process_soil_and_baseflow_to_basin_file(hms_basin_file_path, run)

            # Update Canopy Method condition is met, then modify the file
            if Canopy_Method_Set_To_Simple_Zero == "Yes":
                modify_canopy_method(hms_basin_file_path)

            # Check if Impervious Area Scale is greater than 1 and update the .grid file if necessary
            if run['impervious_area_scale'] > 1:
                dss_file_path = find_impervious_dss_grids(hms_project_directory, run['impervious_area_scale'])
                if dss_file_path:
                    update_impervious_grid_definitions(hms_grid_file_path, "2019_Percent_Impervious", dss_file_path, run['impervious_area_scale'])
                else:
                    # Restore files from backup and exit the script if the appropriate DSS file is not found
                    shutil.copy(hms_project_run_file_backup_path, hms_project_run_file_path)
                    shutil.copy(hms_basin_file_backup_path, hms_basin_file_path)
                    shutil.copy(hms_grid_file_backup_path, hms_grid_file_path)
                    print("Impervious DSS File Not Found. Restoring files and Exiting.")
                    exit()

            # Pause then execute HMS
            time.sleep(5)

            cmd = ['cmd', '/c', hms_compute_bat_path, os.path.join(hms_project_directory, hms_project_name + '.hms'), hms_run_name]
            print(f"Executing HMS with command: {' '.join(cmd)}")

            # Execute HMS
            subprocess.run(cmd)
            
            # HEC-HMS creates a log file with the hms_run_name with a .log extension
            # To avoid overwriting the log file, we rename it to include the run number and the suffix (matching the DSS file name with a .log extension)
            logfile_path = os.path.join(hms_project_directory, hms_run_output_dss.replace('.dss', '.log'))

            # Remove existing log file, if it exists
            if os.path.exists(logfile_path):
                os.remove(logfile_path)

            # Now rename the default HMS log file to match DSS file naming convention
            os.rename(os.path.join(hms_project_directory, hms_run_name + '.log'), logfile_path) if os.path.exists(os.path.join(hms_project_directory, hms_run_name + '.log')) else print(f"The file {os.path.join(hms_project_directory, hms_run_name + '.log')} does not exist.")
            print(f"Renamed log file to {logfile_path}")
            
                
            # Restore original HMS files
            shutil.copy(hms_basin_file_backup_path, hms_basin_file_path)
            print(f"Restored {hms_basin_file_backup_path} to {hms_basin_file_path}")
            shutil.copy(hms_grid_file_backup_path, hms_grid_file_path)
            print(f"Restored {hms_grid_file_backup_path} to {hms_grid_file_path}")
            print("")
        else:
            print(f"Run Output DSS file for Run {current_run_number} already exists.")
            print("")

    print ("All Runs Completed")

    # Delete the lock file for file backup and restoration
    if os.path.exists(lock_file_path):
        os.remove(lock_file_path)
        print("Lock file deleted.")
    else:
        print("Lock file does not exist.")

        
    print(f"All Runs Completed for {run_name}")
    
print ("All Events Processed")
